# DeepLabCut

*This notebook is based off of the outdated multi-animal GoogleColab tutorial notebook on the DeepLabCut website. It has been edited and updated with fixes to make it work*
# ***If you plan to use this script please cite me***
***Daniel Anthony San Miguel Jr., PhD Student at The University of Texas at Austin, College of Pharmacy, Division of Pharmacology & Toxicology, https://github.com/dsanmiguel/DeepLabCut-Scripts/***

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=200w)

https://github.com/DeepLabCut/DeepLabCut

### This notebook demonstrates the necessary steps to use DeepLabCut for your own project and illustrates how to use a single animal project with a Google Colab installed version of DeepLabCut to:

- create a single-animal training set
- train a network
- evaluate a network
- analyze novel videos
- train additional videos on trained model
- assemble animals and tracklets
- create quality check plots

### *This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to Google Drive already*


## **First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "T4 GPU"**

## This first code cell below installs all the dependencies needed to run DeepLabCut in Google Colab as well as DeepLabCut itself (no GUI available in Colab).
## ***This takes about 15-20 minutes to install and set up everything before any other cells below can be ran***

In [1]:
# Upgrade python package manager
!pip3 install --upgrade pip

# Install deeplabcut
!pip3 install -qqq deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

# Install deeplabcut google colab specific things
!pip3 install -qqq --upgrade scikit-image
!pip3 install -qqq pickle5
!pip3 install -qqq torch

# this is so the next line when installing updated cuda doesn't ask for keyboard interaction
!DEBIAN_FRONTEND=noninteractive apt-get install keyboard-configuration

# Update cuda to latest version
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb
!sudo dpkg -i cuda-keyring_1.1-1_all.deb
!sudo apt-get update
!sudo apt-get -y install cuda

# This is just to check what version of cuda we have now to make sure it installed correctly
#!nvcc --version

# This line below is just to search for the name of specific libcudnn8 package to then install in the following line
#!apt-cache policy libcudnn8

# This installs the updated version of libcudnn8 we need that we got from the output above
!sudo apt install --allow-change-held-packages libcudnn8=8.9.5.29-1+cuda12.2

# Add newly installed cuda version to PATH
!export PATH=/usr/local/cuda-12.2/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda-12.2/lib64:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-12.2/include:$LD_LIBRARY_PATH
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI/lib64

# Downgrade tensorflow to older version otherwise deeplabcut.train_network won't work later
!pip3 uninstall -y -qqq tensorflow
!pip3 install -qqq tensorflow==2.8.0

# Install other dependency packages
!pip3 install -qqq segmentation-models
!pip3 install -qqq pycocotools
!pip3 install -qqq opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.3/296.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [ ]:
import deeplabcut
import os
import shutil

# Now, let's link to your Google Drive. Run this next cell and follow the authorization instructions.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


- This code assumes you first locally installed DeepLabCut, created a project, extracted and labeled frames already.

- Be sure to "check Labels" to confirm you are happy with your data as these frames are the only thing that is used to train your network. You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, change directory to the DLC project folder you created previously named "Project-YourName-TheDate"

# **First cell you will need to edit to fit your project**

In [ ]:
# The base path for Google Drive is "/content/drive/MyDrive/"
# My folder "JP23A Run 1 Phase III-Daniel-2023-05-10" is in the subfolder "DeepLabCut" in my Google Drive so that's why I added "DeepLabCut/" to this.
# Make sure there is a backslash / at the end of each of these path variables
google_drive_path = "/content/drive/MyDrive/DeepLabCut/"

# set project_name variable to name of project folder
project_name = 'myprojectfoldername'

## ***If you get an error from running the cell below then you did not set the folder location correctly in the cell above.***

In [ ]:
# This sets the full path location of the project folder in Google Drive to tell Google Colab where to store output later
project_folder = google_drive_path + project_name

# move to directory where project folder is located
os.chdir(project_folder)

# verify you are in the correct location
os.getcwd()

'/content/drive/MyDrive/DeepLabCut/JP23A Run 1 Phase III-Daniel-2023-05-10'

## **Second cell that needs editing**


In [ ]:
# The name of your config file, by default it's named config.yaml unless you changed it.
path_config_file = project_folder + 'config.yaml'

# Enter Phase for these videos, this will serve as the subfolder name for storing analysis output for this data later
# If first time running, you could name it something like "Run1-Phase1" or anything else of your choosing
this_phase = 'myfirstphase'

## **The rest of the code below should be fine to run as is *up until the section "Analyzing New Videos"* where the cells in that section will need to be edited if you decide to label new videos based on this model.**

In [ ]:
# Set path of the folder of videos to analyze based on project_folder path
videofile_path = [project_folder + 'videos/' + this_phase]

# Which shuffle do you want to create and train? Edit if needed, 1 is the default.
shuffle = 1

# This will be where the results for this phase will be stored later
analysis_destination_path = project_folder + 'results/' + this_phase

##*This cell is only necessary to run the first time when creating the project, so if second time running and you get an error that says "folders already exist," please ignore.*



In [ ]:
# Create a results folder within project directory to store output data
os.mkdir('results')

# Create location in results folder where output data will be stored
os.mkdir(analysis_destination_path)

# Create folder where future new video folders can be stored in case we want to use this model on new videos not included in first run
new_videos_folder = project_folder + 'new-videos/'
os.mkdir(new_videos_folder)

#**Start training**
This function trains the network for a specific shuffle of the training dataset.
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/

## Useful information for deeplabcut.create_training_dataset()

***Do not*** run this a <u>**second**</u> time if only increasing max_iters for deeplabcut.train_network(). Only rerun this line again if you edited the config to add more videos, extract more frames, or label more frames.

In [ ]:
deeplabcut.create_training_dataset(path_config_file,
                                   net_type='resnet_50',
                                   augmenter_type = 'imgaug',
                                   Shuffles = [shuffle],
                                   windows2linux = False)

## Useful information for deeplabcut.train_network()

When training for a <u>**second**</u> time, **if** the 'max_iters' variable is set to the same value as the last snapshot iteration number, this will cause an error. This happens because it actually now starts from the iteration of the last snapshot.

For example, if you originally trained it with 200,000 iterations and now want another 200k iterations to improve accuracy, then change max_iters to 400000

In [ ]:
deeplabcut.train_network(path_config_file,
                         shuffle = shuffle,
                         displayiters = 100,
                         saveiters = 1000,
                         maxiters = 75000,
                         allow_growth = True)

### *This code assumes videos are in mp4 format so if not please edit videotype parameter here and in the following cells*

In [ ]:
deeplabcut.analyze_videos(path_config_file,
                          videofile_path,
                          shuffle = shuffle,
                          save_as_csv = True,
                          destfolder = analysis_destination_path,
                          videotype = 'mp4')

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is originally saved in the same directory where the original video resides but we move it to the labeled-videos folder in the main project folder in the following cell.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path,
                                destfolder = analysis_destination_path,
                                videotype = 'mp4',
                                save_frames = False)

In [ ]:
# Create location for labeled-videos
labeled_videos_folder = project_folder + 'labeled-videos/'
os.mkdir(labeled_videos_folder)

# Organize output data into folders by file type
os.chdir(analysis_destination_path)
these_labeled_videos = labeled_videos_folder + this_phase

os.mkdir('CSVs')
os.mkdir('h5')
os.mkdir('pickle')
os.mkdir(these_labeled_videos)

all_analysis_files = os.listdir()

for f in all_analysis_files:
    if f.endswith('.csv'):
        shutil.move(f, 'CSVs')
    elif f.endswith('.h5'):
        shutil.move(f, 'h5')
    elif f.endswith('.pickle'):
        shutil.move(f, 'pickle')
    elif f.endswith('.mp4'):
        shutil.move(f, these_labeled_videos)

***If these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train!***

# **Analyzing New Videos**
This function analyzes the new videos. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in CSV, h5, and pickle format file in the analysis_destination_path.

##**Set "this_phase" variable to the name of the folder of videos within the new-videos folder you want to analyze. This should be the only variable you need to change to label new videos and is the name of the folder within new-videos folder**

In [ ]:
this_phase = 'mynewphase'

In [ ]:
print("Analyzing new videos")
new_videos_folder = project_folder + 'new-videos/'
new_videofile_path = [new_videos_folder + this_phase]
analysis_destination_path = project_folder + 'results/' + this_phase
os.mkdir(analysis_destination_path)

In [ ]:
deeplabcut.analyze_videos(path_config_file,
                          new_videofile_path,
                          shuffle = shuffle,
                          save_as_csv = True,
                          destfolder = analysis_destination_path,
                          videotype = 'mp4')

deeplabcut.create_labeled_video(path_config_file,
                                new_videofile_path,
                                destfolder = analysis_destination_path,
                                videotype = 'mp4',
                                save_frames = False)

In [ ]:
# Set path for labeled-videos folder
labeled_videos_folder = project_folder + 'labeled-videos/'

# Organize output data into folders by file type
os.chdir(analysis_destination_path)
these_labeled_videos = labeled_videos_folder + this_phase

os.mkdir('CSVs')
os.mkdir('h5')
os.mkdir('pickle')
os.mkdir(these_labeled_videos)

all_analysis_files = os.listdir()

for f in all_analysis_files:
    if f.endswith('.csv'):
        shutil.move(f, 'CSVs')
    elif f.endswith('.h5'):
        shutil.move(f, 'h5')
    elif f.endswith('.pickle'):
        shutil.move(f, 'pickle')
    elif f.endswith('.mp4'):
        shutil.move(f, these_labeled_videos)

# **Evaluating Model**

### This section is <u>***not***</u> necessary for training and labeling and is only if you want to estimate how good the model is doing. I have not cleaned this up so you will need to set variable names appropriately and likely edit some code below before you are able to run successfully
- First, we evaluate the pose estimation performance.

- This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, do **NOT** proceed to tracklet assembly, please consider editing config.yaml to:
  1. Add more videos
  2. Extract more frames for labeling
  3. Increase max_iters in deeplabcut.train_network

***Remember that if you edit the config.yaml file to do 1 or 2, you need to recreate the training set for updated config by rerunning cell in Train Network section above with deeplabcut.create_training_dataset***

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file ,
                            Shuffles = [shuffle],
                            plotting = True)

In [ ]:
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file,
                                 shuffle = shuffle,
                                 Indices=[0])

Optional: Now you have the option to check the raw dections before animals are assembled. To do so, pass a video path:

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
## don't look good, don't proceed with tracking - add more data to your training set and re-train!

#EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/My Drive/myprojectfoldername/videos/myvideo.mp4'


deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

If the resutling video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [ ]:
#Check and edit:
numAnimals = 1 #how many animals do you expect to find?
tracktype= 'skeleton' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional:
#imagine you tracked a point that is not useful for assembly,
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

#bodypart= 'Tail_end'
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
deeplabcut.convert_detections2tracklets(path_config_file,
                                        videofile_path,
                                        videotype = VideoType,
                                        shuffle = shuffle,
                                        overwrite = False)

Using snapshot-52000 for model /content/drive/My Drive/trial3-daniel-2023-03-25/dlc-models/iteration-0/trial3Mar25-trainset95shuffle1
Analyzing all the videos in the directory...
Processing...  /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial3.mp4
Analyzing /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial3DLC_dlcrnetms5_trial3Mar25shuffle1_52000.h5


100%|██████████| 15186/15186 [00:26<00:00, 579.90it/s] 
15186it [00:00, 629861.07it/s]


Processing...  /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial4.mp4
Analyzing /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial4DLC_dlcrnetms5_trial3Mar25shuffle1_52000.h5


100%|██████████| 15186/15186 [00:17<00:00, 861.39it/s] 
15186it [00:00, 522811.93it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.


In [ ]:
# split this from code above to see if this is the problem
#deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)
deeplabcut.utils.make_labeled_video.create_video_from_pickled_tracks(Specific_videofile, '/content/drive/My Drive/DeepLabCut/myprojectfolder/results/myfirstphase/pickle/mypicklefile.pickle')

Now let's filter the data to remove any small jitter:

In [ ]:
deeplabcut.filterpredictions(path_config_file,
                                 videofile_path,
                                 shuffle=shuffle,
                                 videotype=VideoType,
                                 track_method = tracktype)

## Create plots of your trajectories:

In [ ]:
#numAnimals = 1 #how many animals do you expect to find?
tracktype= 'box'
shuffle = 1
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial4.mp4 and data.
No unfiltered data file found in /content/drive/My Drive/trial3-daniel-2023-03-25/videos for video Trial4 and scorer DLC_dlcrnetms5_trial3Mar25shuffle1_52000 and box tracker.
No detection data found in /content/drive/My Drive/trial3-daniel-2023-03-25/videos for video Trial4, scorer DLC_dlcrnetms5_trial3Mar25shuffle1_52000, and tracker box
Loading  /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial3.mp4 and data.
No unfiltered data file found in /content/drive/My Drive/trial3-daniel-2023-03-25/videos for video Trial3 and scorer DLC_dlcrnetms5_trial3Mar25shuffle1_52000 and box tracker.
No detection data found in /content/drive/My Drive/trial3-daniel-2023-03-25/videos for video Trial3, scorer DLC_dlcrnetms5_trial3Mar25shuffle1_52000, and tracker box
Plots could not be created for /content/drive/My Drive/trial3-daniel-2023-03-25/videos/Trial4.mp4,/co

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path,
                                shuffle=shuffle,
                                color_by="individual",
                                videotype=VideoType,
                                save_frames=False,
                                filtered=True)